# **Data Preparation Pipeline**

In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY_ZK78")
os.environ['HUGGINGFACEHUB_ACCESS_TOKEN'] = userdata.get("HUGGINGFACEHUB_ACCESS_TOKEN")
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

In [161]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
!pip -q install langchain langchain-pinecone langchain-google-genai openai-agents langchain-community tiktoken python-dotenv pypdf langchain-huggingface sentence-transformers pinecone unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.1/948.1 kB 38.9 MB/s eta 0:00:00
   ━━━

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI,GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.retrievers import MultiQueryRetriever
import os

In [132]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [133]:
import time

index_name = "semester-books"

# if index_name not in pc.list_indexes():
#     pc.create_index(
#         index_name,
#         dimension=384,
#         metric='cosine',
#         spec=ServerlessSpec(cloud='aws', region='us-east-1')
#     )
#     # wait for index to be initialized
#     while not pc.describe_index(index_name).status.ready:
#         time.sleep(1)


# **OpenAI Agents SDK Testing**

In [152]:
from agents import Agent,Runner,OpenAIChatCompletionsModel
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key = os.getenv("GOOGLE_API_KEY"),
)

model = OpenAIChatCompletionsModel(
    model="gemini-1.5-flash",
    openai_client = external_client,
)

In [153]:
agent:Agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=model,
)

response = Runner.run_sync(agent,"Hi")
print(response.final_output)

Hi there! How can I help you today?



In [136]:
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
chat_model.invoke("HI")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--d341d95e-5072-4437-8710-7c9ab4bfbcb9-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

# **Step 1a → Multi-Document Ingestion (per subject)**

## **Linear Algebra**

In [20]:
loader1 = PyPDFLoader("/content/Linear_Algebra.pdf")
docs1 = loader1.load()

In [21]:
len(docs1)

579

In [22]:
docs1[238]

Document(metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'HELIOS pdfcat', 'creationdate': '2015-01-21T14:42:45+05:30', 'codemantra, llc': 'http://www.codemantra.com', 'moddate': "D:20240310035050Z00'00", 'author': '', 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States', 'title': '0321982630.pdf', 'source': '/content/Linear_Algebra.pdf', 'total_pages': 579, 'page': 238, 'page_label': '222'}, page_content='222 CHAPTER 4 Vector Spaces\nIt follows that\n\x8c u C w \x8dB D\n2\n64\nc1 C d1\n:::\ncn C dn\n3\n75 D\n2\n64\nc1\n:::\ncn\n3\n75 C\n2\n64\nd1\n:::\ndn\n3\n75 D \x8c u \x8dB C \x8c w \x8dB\nSo the coordinate mapping preserves addition. Ifr is any scalar, then\nru D r.c1b1 C \x01 \x01 \x01 C cnbn/ D .rc1/b1 C \x01 \x01 \x01 C .rcn/bn\nSo\n\x8c ru \x8dB D\n2\n64\nrc1\n:::\nrcn\n3\n75 D r\n2\n64\nc1\n:::\ncn\n3\n75 D r\x8c u \x8dB\nThus the coordinate mapping also preserves sc

## **Discrete Structures**

In [23]:
loader2 = PyPDFLoader("/content/Discrete mathematics and its applications-BY Kenneth H. Rosen -McGraw-Hill (2013).pdf")
docs2 = loader2.load()

In [24]:
len(docs2)

1070

In [25]:
docs2[178]

Document(metadata={'producer': 'PDFTron PDFNet, V7.10742', 'creator': 'PyPDF', 'creationdate': '2013-08-12T04:58:59+00:00', 'moddate': '2020-04-24T16:00:45+00:00', 'subject': 'TeX output 2011.05.13:1021', 'author': 'Christina Thiele (Carleton CA) 3070 2000 May 31 13:44:37', 'title': 'C:\\rosen4\\FRONT-7T.dvi', 'source': '/content/Discrete mathematics and its applications-BY Kenneth H. Rosen -McGraw-Hill (2013).pdf', 'total_pages': 1070, 'page': 178, 'page_label': '179'}, page_content='2.4 Sequences and Summations 159\nEXAMPLE8 Suppose that { a\nn\n} is the sequence of integers deﬁned by a\nn\n= n ! , the value of the factorial\nfunction at the integer n , where n = 1 , 2 , 3 ,... . Because n ! = n((n − 1 )(n − 2 )... 2 · 1 ) =\nn(n − 1 ) ! = na\nn − 1\n, we see that the sequence of factorials satisﬁes the recurrence relation\na\nn\n= na\nn − 1\n, together with the initial condition a\n1\n= 1.\n▲\nWe say that we have solved the recurrence relation together with the initial conditions wh

## **Calculas & Analytical Geometry**

In [163]:
loader3 = PyPDFLoader("/content/George B. Thomas Jr., Maurice D. Weir, Joel R. Hass - Thomas' Calculus-Pearson (2014).pdf")
docs3 = loader3.load()

In [166]:
len(docs3)

1191

In [167]:
docs3[374]

Document(metadata={'producer': 'Foxit PDF SDK DLL 3.1 - Foxit Software', 'creator': 'Adobe InDesign CS5 (7.0)', 'creationdate': '2015-11-19T23:42:40+05:30', 'moddate': '2016-08-20T17:50:16+02:00', 'author': 'George B. Thomas', 'title': 'Thomas’ Calculus', 'spdf': '1127', 'enhanced': 'By PDF Enhancer 3.5.6412/Unix', 'source': "/content/George B. Thomas Jr., Maurice D. Weir, Joel R. Hass - Thomas' Calculus-Pearson (2014).pdf", 'total_pages': 1191, 'page': 374, 'page_label': '360'}, page_content='THEOREm 2— pappus’s Theorem for Surface a reas If an arc of a smooth \nplane curve is revolved once about a line in the plane that does not cut through \nthe arc’s interior, then the area of the surface generated by the arc equals the \nlength L of the arc times the distance traveled by the arc’s centroid during the \nrevolution. If \nr is the distance from the axis of revolution to the centroid, then\n S = 2prL. (11)\nThe proof we give assumes that we can model the axis of revolution as the x-ax

# **Step 1b → Subject-Aware Text Splitting**


## **Linear Algebra**

In [26]:
PINECONE_LIN = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings,
    namespace="linear_algebra"
)

In [27]:
splitter1 = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks1 = splitter1.split_documents(docs1)

In [28]:
len(chunks1)

1968

In [29]:
chunks1[238]

Document(metadata={'producer': 'xdvipdfmx (0.7.9)', 'creator': 'HELIOS pdfcat', 'creationdate': '2015-01-21T14:42:45+05:30', 'codemantra, llc': 'http://www.codemantra.com', 'moddate': "D:20240310035050Z00'00", 'author': '', 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States', 'title': '0321982630.pdf', 'source': '/content/Linear_Algebra.pdf', 'total_pages': 579, 'page': 75, 'page_label': '59'}, page_content='The proof of the next theorem is similar to the solution of Example 3. Details are given\nat the end of this section.\nT H E O R E M 7Characterization of Linearly Dependent Sets\nAn indexed set S D fv1; : : : ; vpg of two or more vectors is linearly dependent if\nand only if at least one of the vectors inS is a linear combination of the others. In\nfact, ifS is linearly dependent and v1 ¤ 0, then somevj (with j > 1 ) is a linear\ncombination of the preceding vectors, v1; : : :

In [ ]:
vector_store1 = PINECONE_LIN.add_documents(chunks1,namespace="linear_algebra")

## **Discrete Structures**

In [30]:
PINECONE_DIS = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings,
    namespace="discrete_structures"
)

In [31]:
splitter2 = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks2 = splitter2.split_documents(docs2)

In [32]:
len(chunks2)

4869

In [34]:
chunks2[338]

Document(metadata={'producer': 'PDFTron PDFNet, V7.10742', 'creator': 'PyPDF', 'creationdate': '2013-08-12T04:58:59+00:00', 'moddate': '2020-04-24T16:00:45+00:00', 'subject': 'TeX output 2011.05.13:1021', 'author': 'Christina Thiele (Carleton CA) 3070 2000 May 31 13:44:37', 'title': 'C:\\rosen4\\FRONT-7T.dvi', 'source': '/content/Discrete mathematics and its applications-BY Kenneth H. Rosen -McGraw-Hill (2013).pdf', 'total_pages': 1070, 'page': 74, 'page_label': '75'}, page_content='of the call did not put on a special list was billed.\n42. Express each of these system speciﬁcations using predi-\ncates, quantiﬁers, and logical connectives.\na) Every user has access to an electronic mailbox.\nb) The system mailbox can be accessed by everyone in\nthe group if the ﬁle system is locked.\nc) The ﬁrewall is in a diagnostic state only if the proxy\nserver is in a diagnostic state.\nd) At least one router is functioning normally if the\nthroughput is between 100 kbps and 500 kbps and\nthe prox

In [ ]:
vector_store2 = PINECONE_DIS.add_documents(chunks2,namespace="discrete_structures")

## **Calculas & Analytical Geometry**

In [182]:
PINECONE_CAL = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings,
    namespace="calculas_&_analytical_geometry"
)

In [183]:
splitter3 = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks3 = splitter3.split_documents(docs3)

In [184]:
len(chunks3)

4400

In [185]:
chunks3[128]

Document(metadata={'producer': 'Foxit PDF SDK DLL 3.1 - Foxit Software', 'creator': 'Adobe InDesign CS5 (7.0)', 'creationdate': '2015-11-19T23:42:40+05:30', 'moddate': '2016-08-20T17:50:16+02:00', 'author': 'George B. Thomas', 'title': 'Thomas’ Calculus', 'spdf': '1127', 'enhanced': 'By PDF Enhancer 3.5.6412/Unix', 'source': "/content/George B. Thomas Jr., Maurice D. Weir, Joel R. Hass - Thomas' Calculus-Pearson (2014).pdf", 'total_pages': 1191, 'page': 40, 'page_label': '26'}, page_content='sin2 u … u2  and  (1 - cos u)2 … u2.\nBy taking square roots, this is equivalent to saying that\n0 sin u0 … 0 u0   and  0 1 - cos u0 … 0 u0 ,\nso\n- 0 u0 … sin u … 0 u0   and  - 0 u0 … 1 - cos u … 0 u0 .\nThese inequalities will be useful in the next chapter.\nTransformations of Trigonometric Graphs\nThe rules for shifting, stretching, compressing, and reflecting the graph of a function sum-\nmarized in the following diagram apply to the trigonometric functions we have discussed \nin this section.\

In [186]:
vector_store3 = PINECONE_CAL.add_documents(chunks3,namespace="calculas_&_analytical_geometry")

# **Step 2 → Retrieval**

## **Linear_Algebra**

In [101]:
similarity_retriever_lin = PINECONE_LIN.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [138]:
multiquery_retriever_lin = MultiQueryRetriever.from_llm(
    retriever=PINECONE_LIN.as_retriever(search_type="mmr",search_kwargs={"k":1,"lambda_mul":0.8}),
    llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash"),
)

In [139]:
multiquery_retriever_lin

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x78bf43b2f920>, search_type='mmr', search_kwargs={'k': 1, 'lambda_mul': 0.8}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is\n    to generate 3 different versions of the given user\n    question to retrieve relevant documents from a vector  database.\n    By generating multiple perspectives on the user question,\n    your goal is to help the user overcome some of the limitations\n    of distance-based similarity search. Provide these alternative\n    questions separated by newlines. Original question: {question}')
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeS

In [140]:
retrieved_docs = multiquery_retriever_lin.invoke("What is linear algebra")

In [141]:
retrieved_docs

[Document(metadata={'author': '', 'codemantra, llc': 'http://www.codemantra.com', 'creationdate': '2015-01-21T14:42:45+05:30', 'creator': 'HELIOS pdfcat', 'moddate': "D:20240310035050Z00'00", 'page': 9.0, 'page_label': 'ix', 'producer': 'xdvipdfmx (0.7.9)', 'source': '/content/Linear_Algebra.pdf', 'title': '0321982630.pdf', 'total_pages': 579.0, 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States'}, page_content='visualized through the geometric intuition developed in Chapter 1. A major achievement\nof this text is that the level of difﬁculty is fairly even throughout the course.\nA Modern View of Matrix Multiplication\nGood notation is crucial, and the text reﬂects the way scientists and engineers actually\nuse linear algebra in practice. The deﬁnitions and proofs focus on the columns of a ma-\ntrix rather than on the matrix entries. A central theme is to view a matrix–vector prod

## **Discrete_Structures**

In [142]:
multiquery_retriever_dis = MultiQueryRetriever.from_llm(
    retriever=PINECONE_DIS.as_retriever(search_type="mmr",search_kwargs={"k":1,"lambda_mul":0.8}),
    llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash"),
)

In [143]:
multiquery_retriever_dis

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x78bf435208c0>, search_type='mmr', search_kwargs={'k': 1, 'lambda_mul': 0.8}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is\n    to generate 3 different versions of the given user\n    question to retrieve relevant documents from a vector  database.\n    By generating multiple perspectives on the user question,\n    your goal is to help the user overcome some of the limitations\n    of distance-based similarity search. Provide these alternative\n    questions separated by newlines. Original question: {question}')
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeS

In [144]:
multiquery_retriever_dis.invoke("What is discrete structures")

[Document(metadata={'author': '', 'codemantra, llc': 'http://www.codemantra.com', 'creationdate': '2015-01-21T14:42:45+05:30', 'creator': 'HELIOS pdfcat', 'moddate': "D:20240310035050Z00'00", 'page': 262.0, 'page_label': '246', 'producer': 'xdvipdfmx (0.7.9)', 'source': '/content/Linear_Algebra.pdf', 'title': '0321982630.pdf', 'total_pages': 579.0, 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States'}, page_content='that are best explained using linear algebra.\nDiscrete-Time Signals\nThe vector space S of discrete-time signals was introduced in Section 4.1. A signal in\nS is a function deﬁned only on the integers and is visualized as a sequence of numbers,\nsay, fykg. Figure 1 shows three typical signals whose general terms are .:7/k, 1k, and\n.\x001/k, respectively.\nyk = .7k\n– 2 – 1012 – 2 – 1012 – 202\nyk = 1k yk = (– 1)k\nFIGURE 1 Three signals in S.\nSECOND REVISED PAGES'),


## **Calculas & Analytical Geometry**

In [202]:
multiquery_retriever_cal = MultiQueryRetriever.from_llm(
    retriever=PINECONE_CAL.as_retriever(search_type="mmr",search_kwargs={"k":3,"lambda_mul":0.5}),
    llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash"),
)

In [194]:
multiquery_retriever_cal

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x78bf374e3a70>, search_type='mmr', search_kwargs={'k': 1, 'lambda_mul': 0.8}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is\n    to generate 3 different versions of the given user\n    question to retrieve relevant documents from a vector  database.\n    By generating multiple perspectives on the user question,\n    your goal is to help the user overcome some of the limitations\n    of distance-based similarity search. Provide these alternative\n    questions separated by newlines. Original question: {question}')
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeS

In [195]:
multiquery_retriever_cal.invoke("Method of Partial Fractions when ƒ(x),g(x) is Proper")

[Document(metadata={'author': 'George B. Thomas', 'creationdate': '2015-11-19T23:42:40+05:30', 'creator': 'Adobe InDesign CS5 (7.0)', 'enhanced': 'By PDF Enhancer 3.5.6412/Unix', 'moddate': '2016-08-20T17:50:16+02:00', 'page': 483.0, 'page_label': '469', 'producer': 'Foxit PDF SDK DLL 3.1 - Foxit Software', 'source': "/content/George B. Thomas Jr., Maurice D. Weir, Joel R. Hass - Thomas' Calculus-Pearson (2014).pdf", 'spdf': '1127', 'title': 'Thomas’ Calculus', 'total_pages': 1191.0}, page_content='8.5\xa0\xa0Integration of Rational Functions by Partial Fractions 469\ngeneral Description of the Method\nSuccess in writing a rational function ƒ(x)>g(x) as a sum of partial fractions depends on \ntwo things:\n●\t The degree of ƒ(x) must be less than the degree of g(x). That is, the fraction must be \nproper. If it isn’t, divide ƒ(x) by g(x) and work with the remainder term. Example 3 of \nthis section illustrates such a case.\n●\t We must know the factors of g(x). In theory, any polynomial

# **Step 3 → Tool Definitions**

In [116]:
from agents import function_tool
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [117]:
def format_docs(retrieved_docs):
    print(f"[Debug] Retrieved documents: {retrieved_docs}") # Added debug print
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text
llm = GoogleGenerativeAI(model="models/gemini-1.5-flash")

## **Linear Algebra**

In [118]:
@function_tool
def answer_from_linear_algebra(query:str)->str:
  """
  Answer questions about linear algebra using structured RAG chain
  """
  print(f"[Debug] answer_from_linear_algebra function call with query {query}")
  parallel_chain = RunnableParallel({
    'context': multiquery_retriever_lin | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
  }
  )

  prompt = PromptTemplate.from_template(
        """You are a linear algebra expert. Answer the question using only the provided context.

        Context: {context}
        Question: {question}

        Answer:"""
    )

  parser = StrOutputParser()

  main_chain = parallel_chain | prompt | llm | parser
  result = main_chain.invoke(query)
  print(f"[Debug] RAG function call with response ***{result}***")
  return result

## **Discrete Structures**

In [119]:
@function_tool
def answer_from_discrete_structures(query:str)->str:
  """
  Answer questions about discrete structures using structured RAG chain
  """
  print(f"[Debug] answer_from_discrete_structures function call with query {query}")
  parallel_chain = RunnableParallel({
    'context': multiquery_retriever_dis | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
  }
  )

  prompt = PromptTemplate.from_template(
        """You are a discrete structures expert. Answer the question using only the provided context.

        Context: {context}
        Question: {question}

        Answer:"""
    )

  parser = StrOutputParser()

  main_chain = parallel_chain | prompt | llm | parser
  result = main_chain.invoke(query)
  print(f"[Debug] RAG function call with response ***{result}***")
  return result

## **Calculas & Analytical Geometry**

In [203]:
@function_tool
def answer_from_calana(query:str)->str:
  """
  Answer questions about calculas and analytical geometry structures using structured RAG chain
  """
  print(f"[Debug] answer_from_calana function call with query {query}")
  parallel_chain = RunnableParallel({
    'context': multiquery_retriever_cal | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
  }
  )

  prompt = PromptTemplate.from_template(
        """You are a discrete structures expert. Answer the question using only the provided context.

        Context: {context}
        Question: {question}

        Answer:"""
    )

  parser = StrOutputParser()

  main_chain = parallel_chain | prompt | llm | parser
  result = main_chain.invoke(query)
  print(f"[Debug] RAG function call with response ***{result}***")
  return result

# **Agentic Rag Final Form**

In [162]:
import asyncio
qa_agent = Agent(
    name="QA Agent",
    instructions="""
  If your first tool call does not return enough context:
1. Reformulate the query into different variations (e.g., synonyms, break into sub-questions).
2. Retry calling the tool with these variations.
3. Only stop retrying when:
   - You have enough content to give a complete answer, OR
   - All query reformulations have been exhausted.

Never respond to the student without making at least 2 attempts at retrieval.
    """,
    tools=[answer_from_linear_algebra,answer_from_discrete_structures,answer_from_calana ],
    # Use OpenAIChatCompletionsModel with the pre-configured external_client
    model=model,
)

async def main():
    agent_question = "From linear algebra, Tell me about the definition Diognalization."

    # Run the agent
    result = await Runner.run(qa_agent, agent_question)

    # Extract and print the final answer
    # print("Agent result:", result)
    print("Agent's answer:", result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

[Debug] answer_from_linear_algebra function call with query What is the definition of Diagonalization in linear algebra?
[Debug] Retrieved documents: [Document(metadata={'author': '', 'codemantra, llc': 'http://www.codemantra.com', 'creationdate': '2015-01-21T14:42:45+05:30', 'creator': 'HELIOS pdfcat', 'moddate': "D:20240310035050Z00'00", 'page': 110.0, 'page_label': '94', 'producer': 'xdvipdfmx (0.7.9)', 'source': '/content/Linear_Algebra.pdf', 'title': '0321982630.pdf', 'total_pages': 579.0, 'universal pdf': 'The process that creates this PDF constitutes a trade secret of codeMantra, LLC and is protected by the copyright laws of the United States'}, page_content='modern uses of linear algebra. Sections 2.6 and 2.7 describe two interesting applications\nof matrix algebra, to economics and to computer graphics.\n2.1 MATRIX OPERATIONS\nIf A is an m \x02 n matrix—that is, a matrix withm rows and n columns—then the scalar\nentry in the ith row and j th column of A is denoted by aij and i

In [204]:
import asyncio
qa_agent = Agent(
    name="QA Agent",
    instructions="""
  If your first tool call does not return enough context:
1. Reformulate the query into different variations (e.g., synonyms, break into sub-questions).
2. Retry calling the tool with these variations.
3. Only stop retrying when:
   - You have enough content to give a complete answer, OR
   - All query reformulations have been exhausted.

Never respond to the student without making at least 2 attempts at retrieval.
    """,
    tools=[answer_from_linear_algebra,answer_from_discrete_structures,answer_from_calana ],
    # Use OpenAIChatCompletionsModel with the pre-configured external_client
    model=model,
)

async def main():
    agent_question = "From calana, what is Method of Partial Fractions when ƒ(x),g(x) is Proper"

    # Run the agent
    result = await Runner.run(qa_agent, agent_question)

    # Extract and print the final answer
    # print("Agent result:", result)
    print("Agent's answer:", result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

[Debug] answer_from_calana function call with query What is Method of Partial Fractions when ƒ(x),g(x) is Proper?
[Debug] Retrieved documents: [Document(metadata={'author': 'George B. Thomas', 'creationdate': '2015-11-19T23:42:40+05:30', 'creator': 'Adobe InDesign CS5 (7.0)', 'enhanced': 'By PDF Enhancer 3.5.6412/Unix', 'moddate': '2016-08-20T17:50:16+02:00', 'page': 483.0, 'page_label': '469', 'producer': 'Foxit PDF SDK DLL 3.1 - Foxit Software', 'source': "/content/George B. Thomas Jr., Maurice D. Weir, Joel R. Hass - Thomas' Calculus-Pearson (2014).pdf", 'spdf': '1127', 'title': 'Thomas’ Calculus', 'total_pages': 1191.0}, page_content='8.5\xa0\xa0Integration of Rational Functions by Partial Fractions 469\ngeneral Description of the Method\nSuccess in writing a rational function ƒ(x)>g(x) as a sum of partial fractions depends on \ntwo things:\n●\t The degree of ƒ(x) must be less than the degree of g(x). That is, the fraction must be \nproper. If it isn’t, divide ƒ(x) by g(x) and work